# Power Outages
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the severity (number of customers, duration, or demand loss) of a major power outage.
    * Predict the cause of a major power outage.
    * Predict the number and/or severity of major power outages in the year 2020.
    * Predict the electricity consumption of an area.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings

# Blackout Durations: How linear regression can help us predict the severity of a blackout.
By Braden Riggs (A15089134) and Mariam Qader (A15173461)


### Introduction
Across America blackouts occur at different times, in different places, and with varying degrees of duration. For different communities’ blackouts can cause immense damage to local industrial and civilian centers depending on their duration. Using the vast data set “Major Power Outage Risks in the U.S.” provided by Purdue University our machine learning model aims to accurately predict the duration of a blackout based on a variety of variables that can help map the disaster’s severity. Outage duration is a continuous variable that can be predicted using linear regression and will be our target variable for this project. To evaluate the effectiveness of our model we chose to use the coefficient of determination (R-squared) to evaluate how effectively our model predicts blackout duration. Through building this model we hope to construct a framework that can help communities better manage during blackouts.

The dataset consists of 1534 blackouts that impacted at least 50,000 customers or caused an unplanned firm load loss of at least 300MW, from 2000 to 2016.


### Baseline Model

To start our baseline model, first we chose to enumerate the categorical columns in the outages dataset so they can be incorporated into our model. To do this we one hot encoded all of the categorical columns so the presence of each category (state, climate, etc.) are represented by 0's or 1's. The categorical columns we are choosing to use are State, NERC region, Climate region, Climate category, Cause Category and Cause Category Detail. We treated all of the columns as nominal for the purposes of our baseline model, and simply put in the one hot encoder in the categorical column pipeline. Also, we need to take care of the null values in the dataset since it is not possible to train a model with null values present. We simply single-value impute all of the categorical values with the string 'null'. This is a reasonable choice because we are just treating the missing values as its own category. After one hot encoding, we will remove all the perfectly correlated one-hot encoded columns that do not add any further information to the model. We do this by adding the method to the pipeline for the categorical entries. 


The quantitative columns we are choosing to use as features in our model are Year, Month, Outage Start Date/Time, Customers Affected, Percentage of residential/commercial and industrial electricity consumption compared to the total electricity consumption in the state, The Percent of residential/commercial and industrial customers in the state, Urban/Rural population details and Land Area details of the location (land/water percentages). 


We also need to account for the null values in the quantitative data. For the null-values in the customers-affected column we chose to single-value impute them with a 0. This is a reasonable choice for now because we will assume no customers were affected because the value is unknown. The other null values appear when the majority of the data for the instance of the outage is unknown. We chose to drop these occurrences before starting because it did not add any extra information for the purposes of predicting outage duration since the actual duration is missing. 


We impute and one-hot encode the dataset using pipelines, and then fit the data into a linear regression. We then use the train-test-split method, and have chosen to train on 75% of the data and test on 25% of the data. This means we are training the model on a portion of the data, and then testing the model on the remaining portion of the data. This helps avoid overfitting our model to the training data. We want to make sure our model is generalized and can be applied to many different data sets. 


The coefficient of determination (R^2 value) we found using our baseline method was alarmingly low, around 10%. This means only approximately 10% of the data can be explained by this model. This is an alarmingly low value, which really does not predict outage duration well at all. This was not sufficient enough for us, so we decided to engineer some of the features to get better results.
### Final Model
 * chose to put 0 instead of null in customers affected
 * chose to drop cause category details ect already have cause cat which is good

### Fairness Evaluation
TODO

# Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

#Import all necessary sklearn methods 
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split

In [2]:
# Load in outages dataset, and clean/drop unneccesary columns
fp = os.path.join('outage.xlsx')
df = pd.read_excel(fp)
df = df.drop([0,1,2,3])
df.columns = df.iloc[0]
df = df.drop([4])
df = df.reset_index().drop(0).reset_index()
document = df.copy()
outages = df[['U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 
       'CLIMATE.CATEGORY','CAUSE.CATEGORY','OUTAGE.DURATION',
       'CAUSE.CATEGORY.DETAIL','YEAR', 'MONTH', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME',
        'CUSTOMERS.AFFECTED', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN',
       'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'POPULATION',
       'POPPCT_URBAN', 'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC',
       'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND',
       'PCT_WATER_TOT', 'PCT_WATER_INLAND']]

#drop all rows where start time is missing 
outages = outages.dropna(subset=['OUTAGE.START.DATE'])  
outages = outages.dropna(subset=['OUTAGE.DURATION'])  
#Make datetime.time object into an integer so it can work in the pipeline
outages['OUTAGE.START'] = pd.to_numeric(outages.apply(lambda r : pd.datetime.combine(r['OUTAGE.START.DATE'],r['OUTAGE.START.TIME']),1))
#for final model 
outages = outages.drop(['OUTAGE.START.TIME', 'OUTAGE.START.DATE'], axis = 1)

#features used to build model
X = outages.drop(['OUTAGE.DURATION'], axis=1)
#Y = what we are trying to predict, Outage Duration 
y = outages['OUTAGE.DURATION']

In [3]:
#The categorical columns
catcols = ['U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 
       'CLIMATE.CATEGORY','CAUSE.CATEGORY',
       'CAUSE.CATEGORY.DETAIL']
#The quantitative columns 
numcols = ['YEAR', 'MONTH','CUSTOMERS.AFFECTED', 'OUTAGE.START', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN',
       'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'POPULATION',
       'POPPCT_URBAN', 'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC',
       'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND',
       'PCT_WATER_TOT', 'PCT_WATER_INLAND']

# Baseline Model

In [4]:
#Pipeline to impute Nan Values, One Hot encode, and drop perfectly correlated features for categorical columns
cats = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='NULL')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('pca', PCA(svd_solver='full', n_components=0.99))
])
#Column transformer that applies cats, and imputes null values in the numeric columns with 0
ct = ColumnTransformer([
    ('catcols', cats, catcols),
    ('numcols', SimpleImputer(strategy='constant', fill_value=0), numcols)
])

#final pipeline, that applies ct and linear regression
pl = Pipeline([('feats', ct), ('reg', LinearRegression())])

In [5]:
#train, test, split

In [6]:
he = []
for i in range(20):
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
    pl.fit(X_tr, y_tr)
    pred = pl.score(X_ts, y_ts)
    he.append(pred)
base_line_score = str(abs(np.mean(he)*100)) + "%"

In [8]:
base_line_score

'4.171345885818178%'

In [9]:
outages

4,U.S._STATE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,CAUSE.CATEGORY.DETAIL,YEAR,MONTH,CUSTOMERS.AFFECTED,...,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START
0,Minnesota,MRO,East North Central,normal,severe weather,3060,NaN,2011,7,70000,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1309539600000000000
1,Minnesota,MRO,East North Central,normal,intentional attack,1,vandalism,2014,5,NaN,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1399833480000000000
2,Minnesota,MRO,East North Central,cold,severe weather,3000,heavy wind,2010,10,70000,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1288123200000000000
3,Minnesota,MRO,East North Central,normal,severe weather,2550,thunderstorm,2012,6,68200,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1340080200000000000
4,Minnesota,MRO,East North Central,warm,severe weather,1740,NaN,2015,7,250000,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1437184800000000000
5,Minnesota,MRO,East North Central,cold,severe weather,1860,winter storm,2010,11,60000,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1289660400000000000
6,Minnesota,MRO,East North Central,cold,severe weather,2970,tornadoes,2010,7,63000,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1279398600000000000
7,Minnesota,MRO,East North Central,normal,severe weather,3960,thunderstorm,2005,6,300000,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1118203200000000000
8,Minnesota,MRO,East North Central,warm,intentional attack,155,sabotage,2015,3,5941,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1426491060000000000
9,Minnesota,MRO,East North Central,normal,severe weather,3621,hailstorm,2013,6,400000,...,15.28,2279,1700.5,18.2,2.14,0.6,91.5927,8.40733,5.47874,1371836340000000000


# Final Model

In [10]:
#The categorical columns
catcols = ['U.S._STATE', 'CLIMATE.REGION','CAUSE.CATEGORY']
clim_cat = ['CLIMATE.CATEGORY']
#The quantitative columns 
numcols = ['YEAR', 'MONTH',
        'ANOMALY.LEVEL',
       'OUTAGE.START.TIME','DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'TOTAL.PRICE', 
           'POPULATION', 'DAY.START', 'IND.CUST.PCT', 'PC.REALGSP.STATE']
           #'DEMAND.LOSS.MW + CUSTOMERS.AFFECTED', 'DAY.START + CUSTOMERS.AFFECTED', 'DEMAND.LOSS.MW + DAY.START'  ]

In [14]:
doc = document.copy()
#Final Model additional Cleaning
#drop all rows where start time is missing 
doc = doc.dropna(subset=['OUTAGE.START.DATE'])  
doc = doc.dropna(subset=['OUTAGE.DURATION']) 
doc = doc.drop(["level_0", "index", "variables", "OBS", "POSTAL.CODE"], axis = 1)
#Adding Start Day
doc["DAY.START"] = [int(x[8:10]) for x in doc["OUTAGE.START.DATE"].astype("str")]
#Enumerating datetime.time object
doc["OUTAGE.START.TIME"] =  [x.hour *3600 + x.minute * 60 + 60 for x in doc["OUTAGE.START.TIME"]]

#drop outage.start.date
doc = doc.drop(["OUTAGE.START.DATE", "CAUSE.CATEGORY.DETAIL" ], axis = 1)
# Drop nulls for "RES Percent"
doc = doc.loc[doc["RES.PERCEN"].isna() != True]
doc = doc.loc[doc["POPDEN_UC"].isna() != True]


doc["bins"] = pd.qcut(doc["OUTAGE.DURATION"], q = 10, labels = [1, 2, 3, 4, 5,6,7,8,9,10])

#features used to build model
X = doc.drop(['OUTAGE.DURATION', "bins"], axis=1)
#Y = what we are trying to predict, Outage Duration 
y = doc['bins']

doc = doc.drop(['COM.SALES',
       'IND.SALES', 'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN',
       'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS',
       'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE',
       'PC.REALGSP.USA', 'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP',
       'TOTAL.REALGSP', 'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'RES.SALES','POPPCT_URBAN', 
                'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC','PCT_LAND',
       'PCT_WATER_TOT', 'PCT_WATER_INLAND','RES.PRICE',
       'COM.PRICE', 'IND.PRICE', 'AREAPCT_URBAN',
       'POPDEN_RURAL', 'AREAPCT_UC','NERC.REGION'], axis = 1)

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
#Pipeline to impute Nan Values, One Hot encode, and drop perfectly correlated features for categorical columns
cats = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='NULL')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('pca', PCA(svd_solver='full', n_components=0.99))
])

#Create Pipeline to scale all the numeric columns
num_trans = Pipeline(steps = [("scaler", StandardScaler()),
                             ('num_imp', SimpleImputer(strategy='constant', fill_value=0))]) 

#ordinal encoder for climate_category
oe = Pipeline([("ord", OrdinalEncoder())])

#Column transformer that applies cats, and imputes null values in the numeric columns with 0
ct = ColumnTransformer([
    ('catcols', cats, catcols),('ordenc', oe, clim_cat),("numcols", num_trans, numcols)
])

#final pipeline, that applies ct and linear regression
pl = Pipeline([('feats', ct),('reg', RandomForestRegressor(n_estimators = 100))])

In [17]:
holder = []
for i in range(1):
    #Split and test the data
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
    #fit the pipeline
    pl.fit(X_tr, y_tr)
    per = pl.score(X_ts, y_ts)
    holder.append(per)
per = np.mean(holder)

In [18]:
final_score = str(abs(per*100)) + "%"
final_score

'50.512910261736586%'

In [19]:
print("baseline score: "+ base_line_score)
print("final score: "+ final_score)
final_score > base_line_score

baseline score: 4.171345885818178%
final score: 50.512910261736586%


True

### Fairness Evaluation

### Summary of Findings

1. State the *prediction problem* you are attempting. State whether the problem is a classification or regression problem. Explain your choice of target variable and evaluation metric (objective).
1. A summary of the baseline model:
    - The number of features, including how many are quantitative, ordinal, and nominal.
    - The model performance (your evaluation metric) and whether you think this it's good or not (and why).
1. A summary of the final, improved model:
    - The features you added and *why* they are good for your data.
    - The model type you chose; the parameters that ended up performing best; the method of model selection used.
1. Evaluate your model for "fairness" on an interesting subset of the data using a permutation test. Justify the parity measure you are using.

**You will be graded on addressing the above bullet points in your summary.** You should have your code included at bottom in a clearly done, commented fashion. Upon reading the summary, it should be easy for the grader to glance down at the code to see the supporting work.

You may keep your code in a `.py` file and import it if you'd like. Just (1) be sure the output is clearly visible in the notebook, and (2) copy and paste the code from the file to the *very* bottom of the notebook.